# Particle Filter (PF) localisation : Comparing the localised PF from Farchi et al., (2018) (LPF) and the local PF with Inverse Distance Weighting PF (IDWPF) from Cantet et al. (2019)
BC, Nov 2021

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Example case

Consider a case with two points 1 and 2 where we observe the change in the Height of Snow (HS, cm) after a precipitation event.

We want to build an analysis for an unobserved point 3 located in the middle of the observed locations, using the LPF and the IDWPF.
An ensemble of simulations is generated by perturbing the amount of HS change and the temperature from an initial (deterministic) simulation.

In [ ]:
## Model
def hs_change(temp,precip):
    '''
    Our model hs_change(precip,temp) is fairly simple:
     - if temp < 0 is negative: hs_change = precip (snowfall)
     - else: hs_change = -0.2 * precip (rainfall induced settling)
    '''
    if temp <0:
        return precip
    else:
        return -0.2 * precip

In [ ]:
#### EXAMPLE PARAMETERS ####
##  setting the parameters of the initial simulation.

# Temperatures (Celsius)
# Here, point 1 is way above the snow line, point 2 is close to the snow line. Point 3 is in between.
base_temp = [-2, 0.25, -0.9]
error_temp = 1  # additive error

# Precips
base_precip = [20, 20, 20] # identical precipitaiton amounts because points are close to each other.
error_precip = 0.5 # multiplicative error of 50%.

# HS changes from the initial deterministic simulation(cm)
# Consistently with the temperature, it snows in both points, with i
base_changes = [hs_change(T,p) for (p,T) in zip(base_precip, base_temp)]
print('simulated HS changes from the initial simulation : ' + str(base_changes))
nmembers = 1000

## Observations

# Observations are roughy consistent with the temperatures of the initial simulation.
temp_obs = np.random.normal(0, error_temp) + np.array(base_temp)
precip_obs = np.random.normal(1,error_precip) * np.array(base_precip)
obs_all = [hs_change(T,p) for (p,T) in zip(precip_obs, temp_obs)]
obs = obs_all[0:2]
obs_verif =  obs_all[2]
sigma = 2 # # a gaussian error of \sigma cm is expected for each obs
print('observed HS changes to be assimilated : ' + str(obs))
print('observed HS changes to be used for verification: ' + str(obs_verif))


In [ ]:
# #### ENSEMBLE ####

pert_precip = np.random.normal(1, error_precip, nmembers)  # classical multiplicative perturbation for 'precipitation'
pert_temp = np.random.normal(0,error_temp, nmembers)       # classical additive perturbation for 'temperature'
ensemble_hs_change = np.array([[hs_change(base_temp[pt] + pert_temp[mb], base_precip[pt] * pert_precip[mb]) for mb in range(nmembers)] for pt in range(3)])

## Assimilation

In [ ]:
def likelihood(obs, mb, sigma):
    return np.exp(-0.5 * (obs - mb)**2/sigma**2)

In [ ]:
# Compute the weights acording to the LPF and the IDWPF.

likelihoods = np.zeros((2, nmembers))
initial_weights = np.ones((2,nmembers))/nmembers # particles have equal weights before assimilation (/!\ plots and CRPS assume equal weights.)
posterior_weights = np.zeros((2,nmembers))/nmembers # weights before normalization.

## A. Compute the weights on pt1 and pt2 separately
for pt in range(2):
    for mb in range(nmembers):
        likelihoods[pt,mb] = likelihood(obs[pt],ensemble_hs_change[pt,mb],sigma)
        posterior_weights[pt,mb] = likelihoods[pt,mb] * initial_weights[pt,mb]
        
## B. LPF assimilation: 
# B.1. multiply the weights before normalization (Eq. 27 from Farchi and Bocquet, 2018)
posterior_weights_all = np.prod(posterior_weights, axis=0)

# B.2 normalize the LPF posterior weights
posterior_weights_all = posterior_weights_all/np.sum(posterior_weights_all)


# C IDWPF assimilation
# C.1. normalize the posterior weights on pt1 and pt2
for pt in range(2):
    posterior_weights[pt,:] = posterior_weights[pt,:]/np.sum(posterior_weights[pt,:])

# C.2 compute the Inverse distance weighting on pt3: average since pt3 is between pt1 and pt2
posterior_weights_idw = np.mean(posterior_weights, axis=0)


In [ ]:
def resampling(weights):
    """
    PF resampling, Kitagawa (1996)
    """
    nmembers = len(weights)
    resample = np.empty(nmembers, dtype = int)
    zweightcumul = np.cumsum(weights)
    zrand = np.random.random() / nmembers
    for ires in range(0, nmembers):
        if zrand <= zweightcumul[0]:  # zweightcumul[0] = poids du mb 1
            resample[ires] = 1
        for rk in range(1, nmembers):
            if (zweightcumul[rk - 1] < zrand) and (zrand <= zweightcumul[rk]):
                resample[ires] = rk + 1

        zrand += 1. / nmembers
    return resample - 1    # -1 for python indexing


In [ ]:
res_all = resampling(posterior_weights_all)
res_idw = resampling(posterior_weights_idw)
res_pt = np.array([resampling(posterior_weights[pt,:]) for pt in range(2)])

analysis_hs_change_all = np.transpose(np.array([ensemble_hs_change[:,sel] for i, sel in enumerate(res_all)]))
analysis_hs_change_idw = np.transpose(np.array([ensemble_hs_change[:,sel] for i, sel in enumerate(res_idw)]))
analysis_hs_change_pt = np.array([[ensemble_hs_change[pt,res_pt[pt,i]] for i in range(nmembers)] for pt in range(2)])

## Evaluation

In [ ]:
def CRPS(ensemble, obs):

    obsCDF = 0
    ensembleCDF = 0
    precPrevision = 0
    integrale = 0
    ensemble = np.sort(ensemble)
    for prevision in ensemble:
        # immediately after obs
        if obsCDF == 0 and obs < prevision:
            integrale += (obs - precPrevision) * (ensembleCDF ** 2)
            integrale += (prevision - obs) * (ensembleCDF - 1) ** 2
            obsCDF = 1.
        # otherwise
        else:
            integrale += ((prevision - precPrevision) * (ensembleCDF - obsCDF) ** 2)
        ensembleCDF += 1. / len(ensemble)
        precPrevision = prevision

    # if obs > all forecasts
    if obsCDF == 0:
        integrale += obs - prevision

    CRPS = integrale
    return CRPS

In [ ]:
%matplotlib notebook
xmin = -20
xmax = 40
bins = np.arange(xmin,xmax,2)
fig, axs = plt.subplots(ncols =3, sharey = True, figsize = (10,4))
for pt in range(3):
    ax = axs[pt]
    ax.plot(bins, likelihood(obs[pt] if pt<2 else obs_verif, bins,sigma)/4, color = 'r',label = 'obs')

    ax.hist(ensemble_hs_change[pt,:], histtype = 'step', lw  = 2,density = True,bins = bins, label = 'prior')
    #ax.hist(analysis_hs_change_pt[pt,:], histtype = 'step', lw  = 1, density = True,bins = bins,label = 'analysis_pt', color = 'k')
    ax.hist(analysis_hs_change_all[pt,:], histtype = 'step', lw  = 2, density = True,bins = bins,label = 'analysis_all')
    ax.hist(analysis_hs_change_idw[pt,:], histtype = 'step', lw  = 2, density = True,bins = bins,label = 'analysis_idw')
    
    ax.legend()
    ax.set_xlabel('hs change (cm) point ' + str(pt+1))
_ = axs[0].set_ylabel('pdf')

In [ ]:
print('CRPS on pt3 before assimilation (cm):     ' + str(CRPS(ensemble_hs_change[2,:],obs_verif)))
print('CRPS on pt3 with LPF assimilation (cm):   ' + str(CRPS(analysis_hs_change_all[2,:],obs_verif)))
print('CRPS on pt3 with IDWPF assimilation (cm): ' + str(CRPS(analysis_hs_change_idw[2,:],obs_verif)))